# Mini Project 2
Sepehr Abbaspour
610398147

At first, we import 'NLTK' library for tokenisation and achieve stop words list.
The two functions needed are called 'wordpunct_tokenize' and 'stopwords', respectively.
os library and distance function from Levenshtein library are also imported to design an output text as Google and calculate Levenshtein distance, respectively.
The libraries chest and disklist are installed and also imported to make the program be able to create and use dictionary and list data structure on disk, respectively.

In [ ]:
import os, nltk
os.system('color')
from nltk.tokenize import wordpunct_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from Levenshtein import distance
from chest import Chest       #To create dictionaries in disk.
from disklist import DiskList #To create lists in disk.

Defining preprocessor function which preprocess the terms in the documents and queries.
In this function the input string is changed in the way of lowering its letters, elimination of the stop words and asymmetric expansion (changing from plural form to singular form in the specific rules).
Tokenisation level is to delete symbolic letters by iterating the full-lowered string.
Some variables are defined to be pointed for the next times.
After that, the terms are devided and assembled to make String be the list of them and each of them gets ready for a possible changing.
The changing contains stop word deletion in which stop words become removed from the list and replaces with null phrase ("").
This replacements exist to avoid changing the length of the list 'String' while iterating it.
After stop words deletion, we check any plural form for each non-stop word term and if its in any plural form, it'll be changed to its singular form according to official plural nouns table rules.
At last the null phrases in the list of edited terms become removed and the final list is returned by the function.

In [ ]:
def preProcessor(String):
    String = wordpunct_tokenize(String.replace(".", "").lower()) #Case Folding & Tokenisations
    null, Consonants, Irregulars, stopWords = "", "qwrtypsdfghjklzxcvbnm", {"children":"child", "feet":"foot", "teeth": "tooth", "mice": "mouse", "people": "person"}, stopwords.words('english')

    for Index in range(len(String)):
        if String[Index] in stopWords: #Stop Words Elimination
            String[Index] = null
            
        #Asymmetric Expansion
        elif String[Index][-3:] == "ies" and String[Index][-4] in Consonants:
            String[Index] = String[Index][:-3] + "y"

        elif String[Index][-3:] == "ves":
            String[Index] = String[Index][:-3] + "f"
            
        elif String[Index][-2:] == "es" and (String[Index][-3] in ["s", "x", "z"] or String[Index][-4:-2] in ["ch, sh"] or (String[Index][-3] == "o" and String[Index][-4] in Consonants)):
            String[Index] = String[Index][:-2]
        
        elif String[Index][-1] == "s":
            String[Index] = String[Index][:-1]
            
        elif String[Index][-3:] == "men":
            String[Index] = String[Index][:-3] + "man"
        
        elif String[Index] in Irregulars.keys():
            String[Index] = Irregulars[String[Index]]
        
    while null in String:
            String.remove(null)      
    return String

Now we're gonna contruct handler functions for boolean and also proximity, individually.

The first funtion handles queries include the word 'and'.
This function removes the boolean and preprocesses the non-boolean words by the means of the defined function 'preProcessor' and then, check some conditions to return the desired list of document IDs.
At first it checks wether the lower boundary of one postings list is equal to the other or not, or the words from input query are both in the documentaries or not. If both of them becomes negative, the function returns empty list as the intersection.
Else it iterates the postings list of the first word in the query and append any document ID which is also in the postings list of the second word in the query.
This function now handles the input queries which doesn't exist in any documentaries.
Accroding to the mapping functions, the document gaps are converted to the document ID from being in Elias Gamma form because each gap has to be added with the previous document ID in integer form to achieve posting lists for the prior boolean operations at last.

In [ ]:
def And(Query):
    Query = preProcessor(Query.replace(" and ", " "))
    if any(Q not in InvertedIndexStruct for Q in Query):
        return []
    
    ID1, ID2 = list(map(GammaDecode, InvertedIndexStruct[Query[0]])), list(map(GammaDecode, InvertedIndexStruct[Query[1]]))
    lengthID1, lengthID2 = len(ID1), len(ID2)
    if lengthID2 >= lengthID1:                                                                                   
        for Index in range(1, lengthID1):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID1, lengthID2):
            ID2[Index] += ID2[Index - 1]
                                                                                           
    else:
        for Index in range(1, lengthID2):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID2, lengthID1):
            ID1[Index] += ID1[Index - 1]
        
    if ID1[-1] < ID2[0] or ID2[-1] < ID1[0]:
        return []
    
    AND = []
    for ID in ID1:
        if ID in ID2:
            AND.append(ID)
    return sorted(AND)

The second funtion handles queries include the word 'not'.
This function removes the boolean and preprocesses the non-boolean words by the means of the defined function 'preProcessor' and then, check some conditions to return the desired list of document IDs.
At first it checks wether the first word in query which hast the same postings list element with what should achieved from this function is in the documentaries or not. If not, it returns the empty list and ignore the remaining of the query.
If it does exist, it iterates the postings list of the first word in the query and append any document ID which doesn't exist in the postings list of the second word in the query which is pinned by 'not' as its prefix.
This function now handles the input queries which doesn't exist in any documentaries.
Accroding to the mapping functions, the document gaps are converted to the document ID from being in Elias Gamma form because each gap has to be added with the previous document ID in integer form to achieve posting lists for the prior boolean operations at last.

In [ ]:
def Not(Query):
    Query = preProcessor(Query.replace(" not ", " "))
    if Query[0] not in InvertedIndexStruct:
        return []
    
    elif Query[1] not in InvertedIndexStruct:
        return InvertedIndexStruct[Query[0]]

    ID1, ID2, NOT = list(map(GammaDecode, InvertedIndexStruct[Query[0]])), list(map(GammaDecode, InvertedIndexStruct[Query[1]])), []
    lengthID1, lengthID2 = len(ID1), len(ID2)
    if lengthID2 >= lengthID1:                                                                                   
        for Index in range(1, lengthID1):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID1, lengthID2):
            ID2[Index] += ID2[Index - 1]
                                                                                           
    else:
        for Index in range(1, lengthID2):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID2, lengthID1):
            ID1[Index] += ID1[Index - 1]

    for ID in ID1:
        if ID not in ID2:
            NOT.append(ID)
    return sorted(NOT)

The third funtion handles queries include the word 'or'.
This function removes the boolean and preprocesses the non-boolean words by the means of the defined function 'preProcessor' and then, check some conditions to return the desired list of document IDs.
At first it checks wether is there any word in the input query in the documentaries or not. If there's none of them, the function returns empty list and shows that the query is out of the documentaries.
If the length of the postings list of any word in the input query is as much as the number of documentaries, it's concludes that the mentioned word appears in all documentaries and it's postings list would be returned as the definition of 'or' operator.
If any conditions isn't satisfied, exist, it iterates the words in documentaries and assemble their postings list together.
This function now handles the input queries which doesn't exist in any documentaries.
Accroding to the mapping functions, the document gaps are converted to the document ID from being in Elias Gamma form because each gap has to be added with the previous document ID in integer form to achieve posting lists for the prior boolean operations at last.

In [ ]:
def Or(Query):
    Query = preProcessor(Query.replace(" or ", " "))
    if all(Q not in InvertedIndexStruct for Q in Query):
        return []

    ID1, ID2, OR = list(map(GammaDecode, InvertedIndexStruct[Query[0]])), list(map(GammaDecode, InvertedIndexStruct[Query[1]])), []
    IDs, lengthID1, lengthID2 = [ID1, ID2], len(ID1), len(ID2)
    if lengthID2 >= lengthID1:                                                                                   
        for Index in range(1, lengthID1):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID1, lengthID2):
            ID2[Index] += ID2[Index - 1]
                                                                                           
    else:
        for Index in range(1, lengthID2):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID2, lengthID1):
            ID1[Index] += ID1[Index - 1]

    for Index in range(2):
        if Query[Index] in InvertedIndexStruct:
            QID = IDs[Index]
            if len(QID) == lengthDocs:
                return QID
            else:
                OR += QID
    return sorted(set(OR))

The last query handler function is Proximity.
This function removes the proximity phrase in ((near/distance)) form and preprocesses the non-boolean words by the means of the defined function 'preProcessor' and then, check some conditions to return the desired list of document IDs.
The first condition is same as the first condition in the function 'And' because the words of the queries definitions. 
If the conditions aren't satisfied, the function collects the document IDs which contain both of the words to calculates their distance and append them to the list called 'PROX', then.
The function iterates PROX and opens files whose ID is mentioned by one of the elements of PROX and calculates distance between the desired words. If the distance becomes less or equal the boundary the user entered, the document ID will be appended to a special list called 'Proximity'.
At last the list gets sorted and returned.
This function now handles the input queries which doesn't exist in any documentaries.
Accroding to the mapping functions, the document gaps are converted to the document ID from being in Elias Gamma form because each gap has to be added with the previous document ID in integer form to achieve posting lists for the prior boolean operations at last.

In [ ]:
def Proximity(Query):
    String = f" near/{Query.split()[1][-1]} "
    Query = preProcessor(Query.replace(String, " "))
    if any(Q not in InvertedIndexStruct for Q in Query):
        return []
    
    Q0, Q1 = Query[0], Query[1]
    ID1, ID2 = list(map(GammaDecode, InvertedIndexStruct[Q0])), list(map(GammaDecode, InvertedIndexStruct[Q1]))
    lengthID1, lengthID2 = len(ID1), len(ID2)
    if lengthID2 >= lengthID1:                                                                                   
        for Index in range(1, lengthID1):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID1, lengthID2):
            ID2[Index] += ID2[Index - 1]
                                                                                           
    else:
        for Index in range(1, lengthID2):
            ID1[Index] += ID1[Index - 1]
            ID2[Index] += ID2[Index - 1]

        for Index in range(lengthID2, lengthID1):
            ID1[Index] += ID1[Index - 1]

    if ID2[-1] < ID1[0] or ID1[-1] < ID2[0]:
        return []

    PROX = []
    for ID in ID1:
        if ID in ID2:
            PROX.append(ID)

    Proximity = []
    for Element in PROX:
        File = open(listDocs[Element - 1], "r")
        Words = preProcessor(File.read())
        if abs(Words.index(Q0) - Words.index(Q1)) <= int(String[-2]):
            Proximity.append(Element)
        File.close()
    return sorted(Proximity)

Now there's bigram converter for each non-stop singular words in documentaries for a great '*' included searching.
It split the complex query by '*' and make bigram indexes of the word and append them in a list called 'gramList'.

In [ ]:
def biGrams(String):
    lessLengthWC, First, Last = len(String) - 1, String[0], String[-1]
    if First != "*":
        gramList = [f"${First}"]
    else:
        gramList = []

    String = String.split("*")
    for Element in String:
        lengthElement = len(Element) - 1
        for Index in range(lengthElement):
            gramList.append(Element[Index : Index + 2])

    if Last != "*":
        gramList.append(f"{Last}$")

    return gramList

The main function is also defined which calls one of the defined query handler functions according to what type of queries the user input.
Now it has the able to handle complex '*' included and miss spelled queries.
Notice that some recent experiments conclude that this form of applying Levenshtein distance calculation doesn't work well.

In [ ]:
def Main(Query):
    Query = Query.split()
    for Index in [0, 2]:
        Word = Query[Index]
        if "*" in Word:
            setBigrams, listSimilars = set(biGrams(Word)), []
            for Vals in listVals:
                setVals = set(Vals)
                lengthSBV, lengthSVB, Boundary = len(setBigrams - setVals), len(setVals - setBigrams), len(Vals) // 2 + 1 #A hand-made good heuristic boundary to find out how similar the bigram index lists are.
                if all(Val in Word for Val in Vals) or all(Val in Vals for Val in Word) or any(length <= Boundary for length in [lengthSBV, lengthSVB]):
                    listSimilars.append(listKeys[listVals.index(Vals)])
    
            if len(listSimilars) > 1: #Just in Case! #Although a basic but good heuristic boundary has been defined, it's good to find the word with the least Levenshtein distance.
                listCounter = []
                for Key in listSimilars:
                    listCounter.append(distance(Word, Key))
                Query[Index] = listKeys[listCounter.index(min(listCounter))]
                    
            else:
                Query[Index] = listSimilars[0]
                    
            print(f"Hah, Gotcha! Why so nervous and neckless to type '{Query[Index]}' correctly?!")
            print()

        elif Word not in BGs:
            listCounter = []
            for Key in BGs:
                if len(Word) <= int(1.25 * len(Key)): #Another hand-made good heuristic boundary!
                    listCounter.append(distance(Word, Key))
                    
            if listCounter != []:
                Query[Index] = listKeys[listCounter.index(min(listCounter))]
                print(f"Did you mean: \033[1;3m{Query[Index]}\033[0m")
                
            else:
                print("ERROR 404 NOT FOUND!")
                print()

    Query = " ".join(Query)
    if "and" in Query:
        return And(Query)
    
    elif "not" in Query:
        return Not(Query)
                
    elif "or" in Query:
        return Or(Query)
    
    else:
        return Proximity(Query) #The stop words are ignored

According to the two next defined function, the program can convert document gaps from integer to Elias Gamma Code, and vice versa.
This function are constructed in conjuction with some bit manipulations.

In [ ]:
def GammaEncode(gapDoc):
    return (bin(gapDoc)[:1:-1] + '0' * (gapDoc.bit_length() - 1))[::-1]

def GammaDecode(GammaCode):
    return int(''.join(reversed([GammaCode])), 2)

The program has the ability of divide contains of the huge disk containing term-ID pair into some blocks and trigger well-known divide and conquer sorting algorithm 'Merge Sort' on them(by calling 'sortBlocks' function). Notice that the whole operations discussed here occures on memmory.
Now the sorted blocks become attached and make the huge list sorted and each term would be linked to the list of its document gaps in Elias Gamma form and make a huge dictionary on disk called 'InvertedIndexStruct'.

In [ ]:
def sortBlocks(Disk): #Merge sorting by blocks in the memmory.
    if len(Disk) > 1:
        #Index is the point where the array is divided into two subarrays.
        Index = len(Disk)//2
        leftBlock, rightBlock = Disk[:Index], Disk[Index:]
        leftLength, rightLength = len(leftBlock), len(rightBlock)

        # Sort the two halves.
        sortBlocks(leftBlock)
        sortBlocks(rightBlock)
        
        leftIndex = rightIndex = Index = 0

        while leftIndex < leftLength and rightIndex < rightLength:
            if leftBlock[leftIndex] < rightBlock[rightIndex]:
                Disk[Index] = leftBlock[leftIndex]
                leftIndex += 1
                
            else:
                Disk[Index] = rightBlock[rightIndex]
                rightIndex += 1
                
            Index += 1
            
        while leftIndex < leftLength:
            Disk[Index] = leftBlock[leftIndex]
            leftIndex += 1
            Index += 1

        while rightIndex < rightLength:
            Disk[Index] = rightBlock[rightIndex]
            rightIndex += 1
            Index += 1

    return Disk

def BSBI(Disk):
    Disk, InvertedIndexStruct, lengthDisk = sortBlocks(Disk), Chest(), len(Disk)
    for Index in range(lengthDisk):
        Present = Disk[Index] 
        presentTerm, presentID = Present[0], Present[1]
        if presentTerm in InvertedIndexStruct:
            InvertedIndexStruct[presentTerm].append(GammaEncode(presentID - previousID))

        else:
            InvertedIndexStruct[presentTerm] = [GammaEncode(presentID)]
            
        previousID = presentID

    return InvertedIndexStruct

Now there's the ability of getting text file names (case-insensivity is considered by the instruction '.lower()') from user and then, append the input file name to a list created for retaining all names of the documents.
This continues till the user enter "Ctrl+d" which causes EOF error but handled by try... except EOFError instruction.

In [ ]:
#Globals!
Disk, BGs, listDocs, lengthDocs = DiskList(), Chest(), [], 0
while True:
    try:
        while True:
            Document = input("Enter your text document: ")
            if ".txt" == Document[-4:].lower() and Document not in listDocs:
                break
        listDocs.append(Document)
        lengthDocs += 1
        
    except EOFError: #Enter Ctrl+d to exit
        print("START!")
        print()
        break

In this part we're going to construct the elementary list on disk.
Select each document by iterating the list of the names of the input text files and then, open the file. Then it preprocesses on the terms of the opened file line by line by iterating the lines of the file and if the line isn't just a new line, the function 'preProcessor' will be called to preprocess whole strings in that line.
Right after the line is preprocessed, the list of the included terms in the line concate with the list of terms of the opened text file.
This list is used to creating the postings list for each terms of the file and appears in the inverted index structure which has the document IDs of each term exists. If the term is occured in some prior investigated text files, the new document ID is appended to the list of previous document IDs for that term. 
After doing all above for an opened file, the file become closed.
After the file iteration, the inverted index structure will be displayed.

In [ ]:
for Index in range(lengthDocs):
    File, Lines = open(listDocs[Index], "r"), []
    for Line in File:
        if Line != "\n":
            Lines += preProcessor(Line)

    if Lines != []:
        Lines = set(Lines)
        for Term in Lines:
            Disk.append((Term, Index + 1))
                        
            if Term not in BGs:
                 BGs[Term] = biGrams(Term) 
    File.close()

print("Unsorted Disk:")
print(Disk[::])
print()

InvertedIndexStruct, BGs, listKeys, listVals = dict(sorted(BSBI(Disk).items())), dict(sorted(BGs.items())), list(BGs.keys()), list(BGs.values())
print("Sorted Disk:")
print(Disk[::])
print()
print("The Gamma-formed Inverted Index:")
print(InvertedIndexStruct)
print()
print("Bi-gram Index Structure:")
print(BGs)
print()

Query time! This block begins by getting boolean, proximity ('NEAR/' form), complex (wild-card) or miss-spelled queries and nothing else (case-insensivity is considered by the instruction '.lower()').
Then it elicit the target terms by calling the function 'Main' which calls the related function based on which type of query is input.
While user answer 'N' to the question whether she wants to terminate the program, the loop continues.

In [ ]:
while True:
    Query = input("What are you looking for? ").lower()
    BooList = [" not ", " and ", " or ", " near/", "*"] #As the default, promixity has the least priority. Star is added!
    while all(BOO not in Query for BOO in BooList):
        Query = input("What are you looking for? Boolean or proximity please. ").lower()

    print()
    print(Main(Query)) #Main is ready to be called.
    print()
    
    End = input("Wanna end this? <Y, N> ")
    while End not in ["Y", "y", "N", "n"]:
        End = input("DON'T WASTE MY TIME! You wanna end this? <Y, N> ")
    if End in "Yy":
        print("Good-bye")
        break
    else:
        print("Goodluck!")
        print()

This program is in an optimal form due to the pointed conditions in the constructions of the defined query handler functions. 